# LOCALIZACIONES DEFINIDAS
Posibles lugares bien situados

In [1]:
london_loc = "51.520624794291884, -0.0870022262169723"

In [2]:
singapore_loc = "1.2862863534171092, 103.84868192263733"

In [3]:
san_francisco_loc = "37.79787988216447, -122.41049370727525"

# API
Importar librerias 

In [4]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import pyjsonviewer
import geopandas 
import shapely
import pymongo as pym
from pymongo import MongoClient,GEOSPHERE
import shapely.geometry as shg

Tokens

In [5]:
load_dotenv()
#Tenemos los tokens guardados en un .env en el visual studio code

True

In [6]:
tk_id = os.getenv("token_client_id")

In [7]:
tk_secret = os.getenv("token_client_secret")

In [8]:
git_tk = os.getenv("github_tk")

Llamar a la api

In [9]:
url = 'https://api.foursquare.com/v2/venues/explore'

# Londres

## Funciones de llamada a la API


In [10]:
def llamada_API(url, coordenadas, query):
    parametros = {
    "client_id": tk_id,
    "client_secret": tk_secret,
    "v": "20180323",
    "ll": f"{coordenadas}",
    "query": f"{query}",
    "limit": 200  
}
    response = requests.get(url, params=parametros).json()["response"]["groups"][0]["items"]
    return response 

In [11]:
def escoger_datos(response):
    name = []
    type_ = []
    lat = []
    lng = []
    for i in response:
        name.append(i["venue"]["name"])
    for i in response:
        type_.append(i["venue"]["categories"][0]["name"])
    for i in response:
        lng.append(i["venue"]["location"]["lng"])
    for i in response:
        lat.append(i["venue"]["location"]["lat"])
        
    response2 = [name , type_, lat, lng]

    return response2

In [12]:
def crear_df(response2):
    df = pd.DataFrame(response2).T
    df1 = df.rename(columns = {0:"Name", 1:"Type", 2:"Latitude", 3:"Longitude"})
    return df1

In [13]:
def get_geometry(df):
    df2 = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.Latitude, df.Longitude))
    return df2

In [14]:
def get_loc(df):
    df3 = df['geometry'].apply(lambda x:shapely.geometry.mapping(x))
    return pd.DataFrame(df3)

In [15]:
def df_final(url, coordenadas, query):
    a  = llamada_API(url, coordenadas, query)
    b = escoger_datos(a)
    c = crear_df(b)
    d = get_geometry(c)
    d["loc"] = get_loc(d)
    df_terminado = d[["Name","Type","Latitude","Longitude","loc"]]
    return df_terminado

In [23]:
df = df_final(url, london_loc, "Nursery School")

In [24]:
df

,Name,Type,Latitude,Longitude,loc
0,Hatching Dragons,Nursery School,51.521599,-0.095356,"{'type': 'Point', 'coordinates': (51.521599411..."
1,Bright Horizons City Child Nursery and Preschool,Nursery School,51.521406,-0.095647,"{'type': 'Point', 'coordinates': (51.5214063, ..."
2,Hatching Dragons,Nursery School,51.517948,-0.096538,"{'type': 'Point', 'coordinates': (51.517948, -..."
3,Bright Horizons Spitalfields Day Nursery and P...,Nursery School,51.520314,-0.074974,"{'type': 'Point', 'coordinates': (51.520313793..."
4,Hopes and Dreams Montessori Nursery School,Nursery School,51.53105,-0.102079,"{'type': 'Point', 'coordinates': (51.531049874..."
5,The Soulful Nutritionist,Nursery School,51.523818,-0.110315,"{'type': 'Point', 'coordinates': (51.5238179, ..."
6,Rosemary Works Big Play School,Nursery School,51.536926,-0.083277,"{'type': 'Point', 'coordinates': (51.536926463..."
7,Montessori by Busy Bees,Nursery School,51.51005,-0.059813,"{'type': 'Point', 'coordinates': (51.5100497, ..."
8,Bright Horizons Tabard Square Nursery and Pres...,Nursery School,51.500406,-0.090973,"{'type': 'Point', 'coordinates': (51.5004061, ..."
9,Montessori by Busy Bees,Nursery School,51.508951,-0.058398,"{'type': 'Point', 'coordinates': (51.5089513, ..."


In [26]:
a = llamada_API(url, london_loc, "Economics")

geometry --> mongo --> 2dsphere --> geoqueries